PROCESS DOCUMENT

In [ ]:
from IPython.display import Markdown
from utils.document_processor import DocumentProcessor
import os

docs = DocumentProcessor()

DATA_PATH = "./data"
extracted_docs = []
for filename in os.listdir(DATA_PATH):
  valid_extensions = ('.pdf', '.docx', '.txt')
  if not filename.lower().endswith(valid_extensions):
    continue # exclude not supported file type
  
  filepath = os.path.join(DATA_PATH, filename)
  with open(filepath, "rb") as f:
    document = f.read()
    result = docs.process_document(document, filename)
    extracted_docs.append(result)
    
content, images, tables, plain_text = extracted_docs[0] #1st file


TEST OLLAMA MODEL

In [3]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama.llms import OllamaLLM

In [4]:
OLLAMA_MODEL = "llama3.2"
COLLECTION_NAME = "ollama_vectore_test"

In [ ]:
template = """
  You are a helpful assistant for text summarization. 
  Only include information that is part of the document. 
  Do not include your own opinion or analysis.

  Document: 
  "{document}"
  Summary:
"""

prompt = ChatPromptTemplate.from_template(template)

model = OllamaLLM(model=OLLAMA_MODEL)

chain = prompt | model

response = chain.invoke({"document": plain_text})
Markdown(response)


TEST OPENAI MODEL

In [ ]:
from dotenv import load_dotenv
import os
import openai
load_dotenv()

openai.api_type = os.getenv("OPENAI_API_TYPE")
openai.api_base = os.getenv("OPENAI_API_BASE")
openai.api_version = os.getenv("OPENAI_API_VERSION")
openai.api_key = os.getenv("OPENAI_API_KEY")

model = "gpt-35-turbo"
response = openai.ChatCompletion.create(
  engine=model,
  messages=[
    {
      "role": "system", 
      "content": """
        You are a helpful assistant for text summarization.
        Only include information that is part of the document. 
        Do not include your own opinion or analysis.
      """
    },
    {
      "role": "user",
      "content": plain_text
    }
  ],
)

Markdown(response.choices[0].message.content)